In [1]:
# 필요한 라이브러리 불러오기 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# uni-secom 데이터 불러오기 
raw_data = pd.read_csv('uci-secom.csv')

In [3]:
# uni-secom 데이터 프레임 확인  
raw_data.head()

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


# 결측치 처리 
한 열의 결측치 비율이 전체 행의 1% 이하(결측치가 20개 이하)인 열만 남기고 나머지 열은 제거


결측치 중앙값으로 대체 

In [4]:
# 열별 결측치 수 판별
null = []
Not_null = []
null = raw_data.isnull().sum()

In [5]:
# 결측치가 없는 열은 제거 안함 
for i in null.index:
    if null[i] == 0:
        Not_null.append(i)
null = null.drop(Not_null)

In [8]:
# 결측치가 20개 이하인 열은 제거 안함  
null_20 = []
for i in null.index:
    if null[i] < 51:
        null_20.append(i)
null = null.drop(null_20)

# 제거되는 열의 수 확인 
len(null)

60

In [7]:
# 결측치가 20개 이상인 열 제거 
data = raw_data.copy()
data = data.drop(null.index,axis=1)

In [8]:
# 열 제거 후, 남은 결측치 중간값으로 대체 
data.replace(np.nan, data.median(), inplace=True)

# 데이터에 결측치가 있는지 확인 
data.isnull().any().any()

False

In [9]:
# 결측치 제거 후 남은 열 
data.columns

Index(['Time', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '577', '582', '583', '584', '585', '586', '587', '588', '589',
       'Pass/Fail'],
      dtype='object', length=489)

# 'Time' 변수 제거
'Time' 변수는 데이터가 기록된 시간을 의미하는 값으로, 반도체 공정의 데이터에서는 시계열적 요소를 고려하지 않아도 되기 때문에 이는 제거 진행

In [10]:
data = data.drop('Time', axis=1)

# 한 열에 하나의 값만 들어있는 열 제거
한 열이 모두 하나의 값을 갖는다면, 행의 특성을 설명하지 못하는 변수임. 

행의 특성을 반영하지 않는 의미없는 변수이므로 제거 진행.

In [11]:
# 한 열에 하나의 값만 들어있는 열 추출  
unique_columns = []
for i in data.columns:
    tmp =  data[i].unique() 
    if len(tmp) == 1:
        unique_columns.append(i)

# 제거되는 열 수 확인 
len(unique_columns)

111

In [12]:
# 한 열에 하나의 값만 들어있는 열 제거 
data = data.drop(unique_columns, axis=1)

In [13]:
# 열 제거 후 데이터 프레임 정보 확인 
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 377 entries, 0 to Pass/Fail
dtypes: float64(376), int64(1)
memory usage: 4.5 MB


In [14]:
# 모델링을 위해 X,y 분리 
X = data.loc[:,'0':'589']
y = data.loc[:,'Pass/Fail']

In [15]:
# 독립변수 확인 
X

,0,1,2,3,4,6,7,8,9,10,...,576,577,582,583,584,585,586,587,588,589
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,97.6133,0.1242,1.5005,0.0162,-0.0034,...,1.6765,14.9509,0.5005,0.0118,0.0035,2.3630,0.0205,0.0148,0.0046,71.9005
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,102.3433,0.1247,1.4966,-0.0005,-0.0148,...,1.1065,10.9003,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,95.4878,0.1241,1.4436,0.0041,0.0013,...,2.0952,9.2721,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,2988.72,2479.90,2199.0333,909.7926,1.3204,104.2367,0.1217,1.4882,-0.0124,-0.0033,...,1.7585,8.5831,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.3967,0.1235,1.5031,-0.0031,-0.0072,...,1.6597,10.9698,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432
5,2946.25,2432.84,2233.3667,1326.5200,1.5334,100.3967,0.1235,1.5287,0.0167,0.0055,...,1.6679,13.7755,0.4949,0.0189,0.0044,3.8276,0.0342,0.0151,0.0052,44.0077
6,3030.27,2430.12,2230.4222,1463.6606,0.8294,102.3433,0.1247,1.5816,-0.0270,0.0105,...,1.1958,8.3645,0.5010,0.0143,0.0042,2.8515,0.0342,0.0151,0.0052,44.0077
7,3058.88,2690.15,2248.9000,1004.4692,0.7884,106.2400,0.1185,1.5153,0.0157,0.0007,...,56.4274,16.0862,0.4984,0.0106,0.0034,2.1261,0.0204,0.0194,0.0063,95.0310
8,2967.68,2600.47,2248.9000,1004.4692,0.7884,106.2400,0.1185,1.5358,0.0111,-0.0066,...,1.3248,14.2892,0.4993,0.0172,0.0046,3.4456,0.0111,0.0124,0.0045,111.6525
9,3016.11,2428.37,2248.9000,1004.4692,0.7884,106.2400,0.1185,1.5381,0.0159,0.0049,...,0.6636,7.4181,0.4967,0.0152,0.0038,3.0687,0.0212,0.0191,0.0073,90.2294


# 데이터 정규화 
데이터를 모델링하기 전, 모든 변수가 동일한 스케일(중요도)를 갖게 하기 위해 정규화 진행  

In [16]:
# 데이터 정규화를 위한 함수 불러오기 
from sklearn.preprocessing import MinMaxScaler

In [17]:
# MinMaxscaling 진행 
minmaxscaler = MinMaxScaler()
X = minmaxscaler.fit_transform(X)
X = pd.DataFrame(X)

# 변수 축소
대부분의 머신러닝 모델은 입력 데이터의 차원이 클 경우, 차원의 저주와 학습 속도가 저하되는 문제를 갖고 있음. 

이를 해결하기 위해 데이터에서 불필요한 Feature를 제거하는 변수 축소 진행.




*** 차원의 저주: 차원이 클수록 저차원일때보다 예측이 더 불안정해지는 문제 ***


변수 축소 방법으로 LGBM 알고리즘을 통한 변수 선택 방법 적용. 

변수 중요도를 명확히 파악 가능해 투명하고 신뢰성 있는 변수 선택이 가능하기 때문에, 트리 기반의 변수 선택 방법인 LGBM 알고리즘 내재 방법 적용. 





In [18]:
feature_name = list(pd.DataFrame(X).columns)

# 뽑고 싶은 최대 변수 수 지정: 55라고 지정한 이유는 여러 값의 num_feats 적용 후, 적은 데이터셋으로도 높은 예측력을 보이는 수로 선택 
num_feats= 55

In [19]:
# LGBM 알고리즘을 통한 변수 선택 
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
embeded_lgb_selector.fit(X, y)

SelectFromModel(estimator=LGBMClassifier(colsample_bytree=0.2,
                                         learning_rate=0.05,
                                         min_child_weight=40,
                                         min_split_gain=0.01, n_estimators=500,
                                         num_leaves=32, reg_alpha=3,
                                         reg_lambda=1),
                max_features=55)

In [20]:
embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

55 selected features


In [21]:
df_lgb = pd.DataFrame(X, columns=embeded_lgb_feature)
X_lgb = df_lgb.copy() 

In [22]:
# 최종 선택된 독립변수 데이터 셋 
X_lgb.head()

,0,19,20,25,31,40,43,52,58,61,...,301,318,321,329,341,356,365,367,368,373
0,0.469231,0.242124,0.797675,0.398104,0.121293,0.319010,0.411986,0.138291,0.221741,0.762021,...,0.043740,0.155453,0.041580,0.014323,0.000000,0.229765,0.150986,0.112244,0.709375,0.151239
1,0.575003,0.238978,0.712274,0.964903,0.139530,0.360565,0.535243,0.151144,0.204983,0.833427,...,0.036447,0.229751,0.039419,0.014288,0.000000,0.115285,0.102807,0.068396,0.753125,0.220339
2,0.308868,0.238104,0.738940,0.989554,0.067069,0.340259,0.408149,0.267903,0.175564,0.727359,...,0.088241,0.172564,0.037459,0.025803,0.244275,0.983876,0.077769,0.050771,0.562500,0.589309
3,0.400385,0.235237,0.724308,0.955045,0.068333,0.340259,0.447084,0.270716,0.168451,0.750309,...,0.042575,0.139495,0.052237,0.016615,0.000000,0.434376,0.055956,0.043312,0.662500,0.152542
4,0.471367,0.234118,0.720752,0.949862,0.073974,1.000000,0.328147,0.084682,0.350179,0.738046,...,0.036276,0.162191,0.024435,0.015655,0.000000,0.138504,0.090288,0.069148,0.068750,0.152542


In [27]:
data_1 = pd.concat([X_lgb,y], axis=1)

In [35]:
data_1.columns

Index([          0,          19,          20,          25,          31,
                40,          43,          52,          58,          61,
                67,          80,          81,          93,          95,
                97,         100,         102,         110,         129,
               132,         144,         147,         151,         155,
               160,         178,         199,         204,         213,
               214,         216,         217,         222,         224,
               227,         228,         231,         234,         238,
               241,         243,         246,         277,         293,
               301,         318,         321,         329,         341,
               356,         365,         367,         368,         373,
       'Pass/Fail'],
      dtype='object')

In [36]:
from collections import Counter
def detect_outliers(df, n, features):
  outlier_indices = []
  for col in features:
    fraud = df[df['Pass/Fail']==1][col]
    Q1 = np.percentile(fraud.values, 25) #low boundary
    Q3 = np.percentile(fraud.values, 75) #high boundary
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR
    outlier_index = fraud[(fraud < Q1 - outlier_step) | (fraud > Q3 + outlier_step)].index
    outlier_indices.extend(outlier_index)
    if outlier_index.size != 0:
      for row in outlier_index:
        if df[col][row] > Q3 + outlier_step:
          df[col][row] = Q3
        else:
          df[col][row] = Q1
  outlier_indices = Counter(outlier_indices)
  multiple_outliers = list(k for k, v in outlier_indices.items() if v > n)
  print('전체 데이터 행에 대한 이상치 행 비율 = ', len(multiple_outliers)/df[0].size*100, '%')

In [37]:
detect_outliers(data_1, 2, data_1.columns)

전체 데이터 행에 대한 이상치 행 비율 =  0.0 %


In [39]:
y = data_1['Pass/Fail']
X = data_1.drop(['Pass/Fail'],axis=1)

# 모델링 
하이퍼 파리미터 튜닝 후 선택된 모델에 보팅 방식의 앙상블을 적용. 

보팅은 오버피팅을 방지해주는 효과가 있어 채택. 


In [41]:
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
def get_clf_eval(y_test,pred, pred_prob):
  acc = accuracy_score(y_test,pred)
  pre = precision_score(y_test,pred)
  re = recall_score(y_test, pred)
  f1 = f1_score(y_test,pred)
  auc = roc_auc_score(y_test,pred_prob)

  return acc,pre,re,f1,auc

In [42]:
def print_clf_eval(y_test,pred, pred_prob):
  confusion = confusion_matrix(y_test,pred)
  acc,pre,re,f1,auc = get_clf_eval(y_test,pred, pred_prob)

  print('=>오차행렬')
  print(confusion)
  print('=============')
  print('정확도(acc) : {0:.4f}, 정밀도(pre) : {1:.4f}'.format(acc,pre))
  print('재현율(re) : {0:.4f}, F1 : {1:.4f}, AUC : {2:.4f}'.format(re,f1,auc))

In [45]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
from sklearn.model_selection import cross_validate
# VotingClassifier
from sklearn.ensemble import VotingClassifier

# ensemble 할 model 정의
models = [
    ('rfc', RandomForestClassifier(random_state=13, n_jobs=-1, n_estimators=1000)),
    ('lgbm', LGBMClassifier(n_estimators=1000, num_leaves=32, n_jobs=-1,boost_from_average=False)),
    ('xgb', XGBClassifier(n_estimators=500, learning_rate = 0.1)),
    ('etc',ExtraTreesClassifier(n_estimators=100, random_state=13)), 
    ('bc', BaggingClassifier(base_estimator=XGBClassifier(),n_estimators=10, random_state=13))
    ]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=13,stratify=y)
soft_vote  = VotingClassifier(models, voting='soft')
soft_vote.fit(X_train, y_train)
pred_proba_1 = soft_vote.predict_proba(X_test)[:,1]
soft_vote_pred = soft_vote.predict(X_test)
auc_test = roc_auc_score(y_test, pred_proba_1)
print('auc_test =', auc_test)
print('')
print_clf_eval(y_test,soft_vote_pred,pred_proba_1)

auc_test = 0.9054119941491955

=>오차행렬
[[292   1]
 [ 16   5]]
정확도(acc) : 0.9459, 정밀도(pre) : 0.8333
재현율(re) : 0.2381, F1 : 0.3704, AUC : 0.9054


In [61]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
from sklearn.model_selection import cross_validate

# VotingClassifier
from sklearn.ensemble import VotingClassifier

# ensemble 할 model 정의
models = [
    ('rfc', RandomForestClassifier(random_state=13, n_jobs=-1, n_estimators=1000)),
    ('lgbm', LGBMClassifier(n_estimators=1000, num_leaves=32, n_jobs=-1,boost_from_average=False)),
    ('xgb', XGBClassifier(n_estimators=500, learning_rate = 0.1)),
    ('etc',ExtraTreesClassifier(n_estimators=100, random_state=13)), 
    ('bc', BaggingClassifier(base_estimator=XGBClassifier(),n_estimators=10, random_state=13))
    ]

# 모델 성능 평가를 위한 Radom_state 지정 
random_state = []
for j in range(1,101): 
    random_state.append(j)
ai = []

# AUC_ROC SCORE 평균 확인 
for rand in random_state:
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state= rand,stratify=y)
  soft_vote  = VotingClassifier(models, voting='soft')
  soft_vote.fit(X_train, y_train)
  pred_proba_1 = soft_vote.predict_proba(X_test)[:,1]
  auc_test = roc_auc_score(y_test, pred_proba_1)
  print('random_state=', rand,'  ' ,'auc_test =', auc_test)
  print('')
  ai.append(float(auc_test))
print(ai)
print(sum(ai)/len(ai))

random_state= 1    auc_test = 0.8803835527385016

random_state= 2    auc_test = 0.8093612871769869

random_state= 3    auc_test = 0.7950593206565904

random_state= 4    auc_test = 0.7932715748415406

random_state= 5    auc_test = 0.7290752478465788

random_state= 6    auc_test = 0.7929465301478954

random_state= 7    auc_test = 0.808711197789696

random_state= 8    auc_test = 0.7549163009913863

random_state= 9    auc_test = 0.7817324882171298

random_state= 10    auc_test = 0.7692182675117828

random_state= 11    auc_test = 0.843165935316106

random_state= 12    auc_test = 0.858768080611084

random_state= 13    auc_test = 0.797822200552576

random_state= 14    auc_test = 0.7965220217779945

random_state= 15    auc_test = 0.8171623598244759

random_state= 16    auc_test = 0.8249634324719649

random_state= 17    auc_test = 0.7822200552575979

random_state= 18    auc_test = 0.8768080611084025

random_state= 19    auc_test = 0.8522671867381765

random_state= 20    auc_test = 0.83178937103